In [1]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_models
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
import numpy as np

Load Data

In [ ]:
daxdata= get_DAX.get()

In [3]:
daxdata

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ret1,ret2,ret3,ret4,ret5,lag_ret1,lag_ret2,lag_ret3,lag_ret4,lag_ret5
Date,,,,,,,,,,,,,,,,,
1988-01-08 00:00:00+01:00,1026.689941,1026.689941,1026.689941,1026.689941,0,0.0,0.0,1.197370,2.034796,3.024763,7.082493,2.116340,1014.469971,1006.010010,996.099976,956.489990,1005.190002
1988-01-11 00:00:00+01:00,987.520020,987.520020,987.520020,987.520020,0,0.0,0.0,-3.889849,-2.692479,-1.855053,-0.865086,3.192645,1026.689941,1014.469971,1006.010010,996.099976,956.489990
1988-01-12 00:00:00+01:00,986.890015,986.890015,986.890015,986.890015,0,0.0,0.0,-0.063817,-3.953666,-2.756296,-1.918870,-0.928903,987.520020,1026.689941,1014.469971,1006.010010,996.099976
1988-01-13 00:00:00+01:00,965.770020,965.770020,965.770020,965.770020,0,0.0,0.0,-2.163287,-2.227104,-6.116953,-4.919583,-4.082157,986.890015,987.520020,1026.689941,1014.469971,1006.010010
1988-01-14 00:00:00+01:00,974.460022,974.460022,974.460022,974.460022,0,0.0,0.0,0.895776,-1.267511,-1.331328,-5.221176,-4.023806,965.770020,986.890015,987.520020,1026.689941,1014.469971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-01 00:00:00+01:00,16834.130859,16915.919922,16821.599609,16859.039062,89451400,0.0,0.0,-0.264911,-0.669800,-0.489173,-0.605267,-0.283605,16903.759766,16972.339844,16941.710938,16961.390625,16906.919922
2024-02-02 00:00:00+01:00,16977.750000,17004.550781,16894.669922,16918.210938,85847300,0.0,0.0,0.350366,0.085454,-0.319434,-0.138807,-0.254901,16859.039062,16903.759766,16972.339844,16941.710938,16961.390625
2024-02-05 00:00:00+01:00,16925.970703,16972.619141,16859.000000,16904.060547,69935700,0.0,0.0,-0.083675,0.266691,0.001779,-0.403109,-0.222482,16918.210938,16859.039062,16903.759766,16972.339844,16941.710938


In [4]:
energydata = get_energy.get()

  0%|          | 0/268 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 268/268 [00:30<00:00,  8.81it/s]


In [5]:
energydata

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0
...,...,...
2024-02-07 13:00:00,69.24350,2
2024-02-07 14:00:00,67.47475,2
2024-02-07 15:00:00,66.21075,2


test model

Feature extraction, use historic market features to predict future returns

In [62]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from xgboost import XGBRegressor

In [175]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

def quantile_loss(quantile):
    def _quantile_loss(y_true, y_pred):
        x = y_true - y_pred
        grad = (x < 0) * (1 - quantile) - (x > 0) * quantile
        hess = np.ones_like(x)
        return grad, hess
    return _quantile_loss
import numpy as np

import numpy as np

def compute_cumulative_log_returns(df):
    """
    Calculate cumulative logarithmic future returns from 1 day ahead to 5 days ahead.
    Cumulates returns from 1-1, 1-2, ..., up to 1-5, labeled as 'log_ret{days}_forward'.
    """
    # Calculate log returns for each day ahead
    for days in range(1, 6):
        df[f'temp_log_ret{days}'] = np.log(df['Close'] / df['Close'].shift(-days))
    
    # Calculate cumulative log returns
    for days in range(1, 6):
        df[f'log_ret{days}_forward'] = df[[f'temp_log_ret{i}' for i in range(1, days + 1)]].sum(axis=1)
    
    # Drop the temporary columns and rows where any of the calculated returns are NaN
    df = df.drop(columns=[f'temp_log_ret{i}' for i in range(1, 6)])
    df.dropna(subset=[f'log_ret{days}_forward' for days in range(1, 6)], inplace=True)
    
    return df


def preprocess_data_for_prediction(df):
    """
    Generate lagged market features and normalize features for prediction.
    """
    df.index = pd.to_datetime(df.index)
    lag_features = ['Open', 'High', 'Low', 'Close']
    for feature in lag_features:
        for lag in range(1, 6):
            df[f'{feature}_lag{lag}'] = df[feature].shift(lag)
    df['Close_MA5'] = df['Close'].rolling(window=5).mean().shift(1)
    df = compute_cumulative_log_returns(df)
    df.drop(columns=['Volume'], inplace=True)  # Assuming 'Volume' is not among the features used for training
    
    # Normalize features
    features = [f'{feat}_lag{j}' for feat in lag_features for j in range(1, 6)] + ['Close_MA5']
    scaler_X = StandardScaler()
    df[features] = scaler_X.fit_transform(df[features])
    
    return df, scaler_X, features

from sklearn.metrics import make_scorer
from xgboost import XGBRegressor

def xgb_quantile_grad_hess(quantile, y_true, y_pred):
    error = y_true - y_pred
    grad = np.where(error > 0, -quantile, -(quantile - 1))
    hess = np.ones_like(y_pred)
    return grad, hess

def train_and_predict(df, features, quantiles):
    """
    Train XGBRegressor models for different quantiles using custom pinball loss and predict future returns.
    """
    quantile_models = {}
    
    for days in range(1, 6):
        X = df[features]
        y = df[f'log_ret{days}_forward']
        for quantile in quantiles:
            model = XGBRegressor(objective=lambda y_true, y_pred: xgb_quantile_grad_hess(quantile, y_true, y_pred),n_estimators=100, max_depth=3, learning_rate=0.1, verbosity=0)

            model.fit(X, y)
            # Storing model
            quantile_models[(days, quantile)] = model
            
    return quantile_models


def predict_for_date(df, forecast_date, features, scaler_X, quantile_models):
    """
    Prepare features for a specific forecast date and predict future returns.
    """
    forecast_date = pd.to_datetime(forecast_date).tz_localize('Europe/Berlin')
    if forecast_date in df.index:
        X_forecast = df.loc[[forecast_date]][features]
        forecast_predictions = {}
        
        for days in range(1, 6):
            forecast_predictions[days] = {}
            for quantile in [0.025, 0.25, 0.5, 0.75, 0.975]:
                model = quantile_models[(days, quantile)]
                forecast_predictions[days][quantile] = model.predict(X_forecast)[0]
                
        return forecast_predictions
    else:
        raise ValueError(f"No data available for forecast_date: {forecast_date}")

quantiles= [0.025, 0.25, 0.5, 0.75, 0.975]
daxdata=get_DAX.get()
daxdata, scaler_X, features = preprocess_data_for_prediction(daxdata)
quantile_models = train_and_predict(daxdata, features,quantiles)

# Specify the forecast date you're interested in
forecast_date = '2024-01-25'

# Predict future returns for the specified forecast date
forecast_predictions = predict_for_date(daxdata, forecast_date, features, scaler_X, quantile_models)

# Display forecast predictions
for day in forecast_predictions:
    print(f"Day {day} predictions:")
    for quantile, prediction in forecast_predictions[day].items():
        print(f"  Quantile {quantile}: {prediction}")


/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is depre

Day 1 predictions:
  Quantile 0.025: -0.08196770399808884
  Quantile 0.25: -0.024039676412940025
  Quantile 0.5: -0.013871826231479645
  Quantile 0.75: 0.058699849992990494
  Quantile 0.975: 0.250027060508728
Day 2 predictions:
  Quantile 0.025: -0.010113532654941082
  Quantile 0.25: -0.027705110609531403
  Quantile 0.5: 0.022891400381922722
  Quantile 0.75: 0.039553042501211166
  Quantile 0.975: 0.2500285506248474
Day 3 predictions:
  Quantile 0.025: -0.013807491399347782
  Quantile 0.25: -0.06185248866677284
  Quantile 0.5: 0.0014709122478961945
  Quantile 0.75: -0.010026020929217339
  Quantile 0.975: 0.2504099905490875
Day 4 predictions:
  Quantile 0.025: -0.016753697767853737
  Quantile 0.25: -0.07109018415212631
  Quantile 0.5: 0.007460929453372955
  Quantile 0.75: 0.03632836788892746
  Quantile 0.975: 0.2516980469226837
Day 5 predictions:
  Quantile 0.025: -0.08750040829181671
  Quantile 0.25: -0.02195444330573082
  Quantile 0.5: 0.03412389010190964
  Quantile 0.75: 0.01850185729

/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is depre

In [176]:
daxdata_with_future_returns=compute_cumulative_log_returns(daxdata)

In [177]:
daxdata_with_future_returns

,Open,High,Low,Close,Dividends,Stock Splits,ret1,ret2,ret3,ret4,...,Close_lag2,Close_lag3,Close_lag4,Close_lag5,Close_MA5,log_ret1_forward,log_ret2_forward,log_ret3_forward,log_ret4_forward,log_ret5_forward
Date,,,,,,,,,,,,,,,,,,,,,
1988-01-08 00:00:00+01:00,1026.689941,1026.689941,1026.689941,1026.689941,0.0,0.0,1.197370,2.034796,3.024763,7.082493,...,NaN,NaN,NaN,NaN,NaN,0.038898,0.078435,0.139605,0.191816,0.266864
1988-01-11 00:00:00+01:00,987.520020,987.520020,987.520020,987.520020,0.0,0.0,-3.889849,-2.692479,-1.855053,-0.865086,...,NaN,NaN,NaN,NaN,NaN,0.000638,0.022909,0.036222,0.072371,0.056687
1988-01-12 00:00:00+01:00,986.890015,986.890015,986.890015,986.890015,0.0,0.0,-0.063817,-3.953666,-2.756296,-1.918870,...,-1.307555,NaN,NaN,NaN,NaN,0.021633,0.034308,0.069818,0.053497,0.060319
1988-01-13 00:00:00+01:00,965.770020,965.770020,965.770020,965.770020,0.0,0.0,-2.163287,-2.227104,-6.116953,-4.919583,...,-1.316732,-1.307643,NaN,NaN,NaN,-0.008958,0.004920,-0.033035,-0.047845,-0.042311
1988-01-14 00:00:00+01:00,974.460022,974.460022,974.460022,974.460022,0.0,0.0,0.895776,-1.267511,-1.331328,-5.221176,...,-1.316880,-1.316823,-1.307724,NaN,NaN,0.022835,-0.006162,-0.012014,0.002478,0.028700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-02 00:00:00+01:00,16977.750000,17004.550781,16894.669922,16918.210938,0.0,0.0,0.350366,0.085454,-0.319434,-0.138807,...,2.412370,2.429367,2.423094,2.428618,2.419275,0.000837,-0.005939,-0.006161,-0.007927,-0.007927
2024-02-05 00:00:00+01:00,16925.970703,16972.619141,16859.000000,16904.060547,0.0,0.0,-0.083675,0.266691,0.001779,-0.403109,...,2.401892,2.413295,2.430274,2.424003,2.417250,-0.007613,-0.008671,-0.011274,-0.011274,-0.011274
2024-02-06 00:00:00+01:00,16965.890625,17049.519531,16871.320312,17033.240234,0.0,0.0,0.761288,0.677613,1.027979,0.763067,...,2.415755,2.402814,2.414198,2.431185,2.415485,0.006555,0.011564,0.011564,0.011564,0.011564
